<a href="https://colab.research.google.com/github/BryanCoboF/optimizacionCosecha/blob/main/sitio/optCosechaPQAntesmodJM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuración Inicial
Llamada a librerias

In [326]:
#Configuracion de librerias
%config IPCompleter.greedy=True
!pip install pulp
import pandas as pd
import numpy as np
import xlrd
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import pulp
from pulp import LpVariable, LpProblem, LpMaximize, LpMinimize
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive



Carga de datos de Excel desde drive

In [327]:

from google.colab import files
from google.colab import drive

drive.mount('/content/drive') # Permite ingresar a drive desde las carpetas de drive

df = pd.read_excel("/content/drive/MyDrive/PB/BaseOpt2023.xlsx") # Carga el excel de datos de proyeccion, si se necesita cambiar el archivo se debe modificar el link



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ingreso de datos deseados


1.   Volumen deseado por cada mes
3.   Densidad deseada por cada mes
2.   Edad minima y maxima deseada


In [328]:
# Volumen deseado cada mes
vecVolDeseado = []
vecVolR11minDeseado = []
vecVolR11maxDeseado = []
vecVolmaxDeseado = []
vecDensminDeseada = []
vecDensmaxDeseada = []

#semanasMes = [4,4,5,4,5,4,4,5,4,4,5,4] #Numero de semanas por cada mes
semanasMes = [0,0,0,0,0,0,0,4,4,4,4,4] #Numero de semanas por cada mes
#Ingreso de volumen anual deseado
volDes = float(input("Ingrese el volumen deseado 2024: "))

# Calculo del volumen deseado para cada mes en base al deseado anual y las semanas que tiene cada mes.
#for i in range(12):
#  vecVolDeseado.append((volDes/52)*semanasMes[i])


# Calculo del volumen deseado para cada mes en base al deseado anual y las semanas que tiene cada mes.
for i in range(12):
  vecVolDeseado.append((volDes/20)*semanasMes[i])

# --------------------------------------------------------------------------------------------------
# Solo usado para 2023 comentar para los siguientes años
vecVolDeseado = [0,0,0,0,0,0,0,2250000,2500000,2500000,2000000,1800000] #Numero de semanas por cada mes
# ---------------------------------------------------------------------------------------------------

print(vecVolDeseado)
print(sum(vecVolDeseado))

#Ingreso de edad deseada
edadMinDes = float(input("Ingrese la edad minima de cosecha : "))
areaMaxCosecha = float(input("Ingrese el area máxima cosechable por sitio cada mes: "))

Ingrese el volumen deseado 2024: 11050000
[0, 0, 0, 0, 0, 0, 0, 2250000, 2500000, 2500000, 2000000, 1800000]
11050000
Ingrese la edad minima de cosecha : 26
Ingrese el area máxima cosechable por sitio cada mes: 150


OBTENCION  DE DATOS DEL EXCEL
Creación de Diccionarios Y VECTORES


In [329]:
# Obtencion de datos de sitio-año de plantacion
colSitioAnio = df['Sitio-AñoPlantación']

# Obtencion de datos area neta
colAreaNeta = df['Sum of Área Neta del Lote (Ha)']

# ------------- Obtencion de datos de edad -----------------
# Recibe data frame con los datos de cada mes del excel
# Crea un diccionario por cada mes con los datos por sitio-lote
#Enero
colEdadEne = df['Sum of Edad pond ENE']
#Febrero
colEdadFeb = df['Sum of Edad pond FEB']
#Marzo
colEdadMar = df['Sum of Edad pond MAR']
#Abril
colEdadAbr = df['Sum of Edad pond ABR']
#Mayo
colEdadMay = df['Sum of Edad pond MAY']
#Junio
colEdadJun = df['Sum of Edad pond JUN']
#Julio
colEdadJul = df['Sum of Edad pond JUL']
#Agosto
colEdadAgo = df['Sum of Edad pond AGO']
#Septiembre
colEdadSep = df['Sum of Edad pond SEP']
#Octubre
colEdadOct = df['Sum of Edad pond OCT']
#Noviembre
colEdadNov = df['Sum of Edad pond NOV']
#Diciembre
colEdadDic = df['Sum of Edad pond DIC']


#--------OBTENCION DE DATOS DE RENDIMIENTO ----------------
# Obtiene los datos de rendimiento del data frame de excel de cada mes proyectado
#Enero
colRendProyEne = df['Sum of Rend pond ENE']
#Febrero
colRendProyFeb = df['Sum of Rend pond FEB']
#Marzo
colRendProyMar = df['Sum of Rend pond MAR']
#Abril
colRendProyAbr = df['Sum of Rend pond ABR']
#Mayo
colRendProyMay = df['Sum of Rend pond MAY']
#Junio
colRendProyJun = df['Sum of Rend pond JUN']
#Julio
colRendProyJul = df['Sum of Rend pond JUL']
#Agosto
colRendProyAgo = df['Sum of Rend pond AGO']
#Septiembre
colRendProySep = df['Sum of Rend pond SEP']
#Octubre
colRendProyOct = df['Sum of Rend pond OCT']
#Noviembre
colRendProyNov = df['Sum of Rend pond NOV']
#Dicimebre
colRendProyDic = df['Sum of Rend pond DIC']

#------- OBTENCION DE DATOS DE ACCESIBILIDAD -------------------
#Enero
colAccesoEne = df['Sum of AccesibilidadENE']
#Febrero
colAccesoFeb = df['Sum of AccesibilidadFEB']
#Marzo
colAccesoMar = df['Sum of AccesibilidadMAR']
#Abril
colAccesoAbr = df['Sum of AccesibilidadABR']
#Mayo
colAccesoMay = df['Sum of AccesibilidadMAY']
#Junio
colAccesoJun = df['Sum of AccesibilidadJUN']
#Julio
colAccesoJul = df['Sum of AccesibilidadJUL']
#Agosto
colAccesoAgo = df['Sum of AccesibilidadAGO']
#Septiembre
colAccesoSep = df['Sum of AccesibilidadSEP']
#Octubre
colAccesoOct = df['Sum of AccesibilidadOCT']
#Novimebre
colAccesoNov = df['Sum of AccesibilidadNOV']
#Diciembre
colAccesoDic = df['Sum of AccesibilidadDIC']

# Obtencion de costo de activo biologico
colCostoActBio = df['Sum of CostoActBio Pond']

# -----------OBTENCION DE COSTO OPERACIONAL ------------------------
#Enero
colCostoOpeEne = df['Sum of Costo Operacional pond ENE']
#Febrero
colCostoOpeFeb = df['Sum of Costo Operacional pond FEB']
#Marzo
colCostoOpeMar = df['Sum of Costo Operacional pond MAR']
#Abril
colCostoOpeAbr = df['Sum of Costo Operacional pond ABR']
#Mayo
colCostoOpeMay = df['Sum of Costo Operacional pond MAY']
#Junio
colCostoOpeJun = df['Sum of Costo Operacional pond JUN']
#Julio
colCostoOpeJul = df['Sum of Costo Operacional pond JUL']
#Agosto
colCostoOpeAgo = df['Sum of Costo Operacional pond AGO']
#Septiembre
colCostoOpeSep = df['Sum of Costo Operacional pond SEP']
#Octubre
colCostoOpeOct = df['Sum of Costo Operacional pond OCT']
#Noviembre
colCostoOpeNov = df['Sum of Costo Operacional pond NOV']
#Diciembre
colCostoOpeDic = df['Sum of Costo Operacional pond DIC']





# ---------CREACION DE MATRICES POR CADA MES DEL AÑO ------------
filas = len(colSitioAnio)  # Número de filas
columnas = 12  # Número de columnas (meses del siguiente año)

#//MATRICES//
# Inicializa las matrices n filas y 12 columnas
#Todos los valores se inicializan  con cero  /// Todas las matrices son del mismo tamaño
matrizEdad = [[0] * columnas for _ in range(filas)]
matrizRend = [[0] * columnas for _ in range(filas)]
matrizRendAux = [[0] * columnas for _ in range(filas)]
matrizAcceso = [[0] * columnas for _ in range(filas)]
matrizCostoOpe = [[0] * columnas for _ in range(filas)]
matrizCostobft = [[0] * columnas for _ in range(filas)]


# -------------Asignacion de valores a cada mes de las matrices---------
# Matriz de Edades por cada mes del 2024
# Matriz  de rendimientos
# Matriz de Accesibilidad
# Matriz de costo operacional

for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    if i == 0:
      matrizEdad[j][i] = colEdadEne[j]
      matrizRend[j][i] = colRendProyEne[j]
      matrizAcceso[j][i] = colAccesoEne [j]

    if i == 1:
      matrizEdad[j][i] = colEdadFeb[j]
      matrizRend[j][i] = colRendProyFeb[j]
      matrizAcceso[j][i] = colAccesoFeb [j]

    if i == 2:
      matrizEdad[j][i] = colEdadMar[j]
      matrizRend[j][i] = colRendProyMar[j]
      matrizAcceso[j][i] = colAccesoMar [j]

    if i == 3:
      matrizEdad[j][i] = colEdadAbr[j]
      matrizRend[j][i] = colRendProyAbr[j]
      matrizAcceso[j][i] = colAccesoAbr [j]

    if i == 4:
      matrizEdad[j][i] = colEdadMay[j]
      matrizRend[j][i] = colRendProyMay[j]
      matrizAcceso[j][i] = colAccesoMay [j]

    if i == 5:
      matrizEdad[j][i] = colEdadJun[j]
      matrizRend[j][i] = colRendProyJun[j]
      matrizAcceso[j][i] = colAccesoJun [j]

    if i == 6:
      matrizEdad[j][i] = colEdadJul[j]
      matrizRend[j][i] = colRendProyJul[j]
      matrizAcceso[j][i] = colAccesoJul [j]

    if i == 7:
      matrizEdad[j][i] = colEdadAgo[j]
      matrizRend[j][i] = colRendProyAgo[j]
      matrizAcceso[j][i] = colAccesoAgo [j]

    if i == 8:
      matrizEdad[j][i] = colEdadSep[j]
      matrizRend[j][i] = colRendProySep[j]
      matrizAcceso[j][i] = colAccesoSep [j]

    if i == 9:
      matrizEdad[j][i] = colEdadOct[j]
      matrizRend[j][i] = colRendProyOct[j]
      matrizAcceso[j][i] = colAccesoOct [j]

    if i == 10:
      matrizEdad[j][i] = colEdadNov[j]
      matrizRend[j][i] = colRendProyNov[j]
      matrizAcceso[j][i] = colAccesoNov [j]

    if i == 11:
      matrizEdad[j][i] = colEdadDic[j]
      matrizRend[j][i] = colRendProyDic[j]
      matrizAcceso[j][i] = colAccesoDic [j]



Creacion del diccionario con todas las edades

RESTRICCION DE EDAD DE COSECHA Y ACCESIBILIDAD

In [330]:
# Si el sitio cumple con los requerimientos de edad y accesibilidad en ese mes se asgina el rendimiento proyectado,
# caso contrario se asigna 0
# Pregunta primero si desea aplicar las restricciones

restEdad = float(input("Desea restringir edad? (SI: Ingrese 1,  NO: Ingrese 0) : "))
restAcceso = float(input("Desea restringir accesibilidad? (SI: Ingrese 1,  NO: Ingrese 0) : "))


if restEdad == 1 and restAcceso == 1:
  # Restricciones completas --- Se corre solo si se ingreso que se desea restringir edad y acceso
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizAcceso[j][i] == 1 and matrizEdad[j][i] >= 26:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad== 0 and restAcceso == 1:
  # Restriccion de acceso unicamente
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizAcceso[j][i] == 1:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad== 1 and restAcceso == 0:
  # Restriccion solo de edad
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizEdad[j][i] >= 26:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad== 0 and restAcceso == 0:
  # Sin restricciones
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      matrizRendAux[j][i] = matrizRend[j][i]

Desea restringir edad? (SI: Ingrese 1,  NO: Ingrese 0) : 1
Desea restringir accesibilidad? (SI: Ingrese 1,  NO: Ingrese 0) : 0


-------------------------------------------

MODELO OPTIMIZACION

------------------------------------------------------------

In [331]:
#****************************** MODELO*********************************************************************************
#-------------Definicion del modelo-----------------
optCosecha = pulp.LpProblem('OptimizacionCosecha',pulp.LpMaximize) #Inicialmente busca minimizar el area de cosecha para obtener el volumen necesario para Enero


# ------------VARIABLE DE DECISION (AREA A COSECHAR)-------------------
area_a_cosechar = [[pulp.LpVariable(f'x{i}_{j}', lowBound=0, cat='Continuous') for j in range(columnas)] for i in range(filas)]


#--------------FUNCION OBJETIVO-------------------------
#optCosecha += sum([rend_EneAux[i]*area_a_cosechar_Ene[i]+rend_FebAux[i]*area_a_cosechar_Feb[i]+ rend_MarAux[i]*area_a_cosechar_Mar[i]+rend_AbrAux[i]*area_a_cosechar_Abr[i]+rend_MayAux[i]*area_a_cosechar_May[i]+rend_JunAux[i]*area_a_cosechar_Jun[i]+rend_JulAux[i]*area_a_cosechar_Jul[i]+rend_AgoAux[i]*area_a_cosechar_Ago[i]+rend_SepAux[i]*area_a_cosechar_Sep[i]+rend_OctAux[i]*area_a_cosechar_Oct[i]+rend_NovAux[i]*area_a_cosechar_Nov[i]+rend_DicAux[i]*area_a_cosechar_Dic[i] for i in colSitioAnio])
optCosecha += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar[i][j] for j in range(columnas) for i in range(filas))


#--------------RESTRICCIONES----------------------------
# 1) -- Restringe las areas a cosechar de cada sitio y mes para que no sean mayores al area neta
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    optCosecha += area_a_cosechar[j][i] >= 0
    optCosecha += area_a_cosechar[j][i] <= colAreaNeta[j]
    optCosecha += area_a_cosechar[j][i] <= areaMaxCosecha

# 2) -- Restringe que la suma del area a cosecha de cada sitio en todo el año sea menor o igual al area neta
for j in range(filas): # recorre filas
  optCosecha += sum(area_a_cosechar[j]) <= colAreaNeta[j]

# 3) Restriccion de volumen anual
optCosecha += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar[i][j] for j in range(columnas) for i in range(filas)) <= 11050000

# 4) Restringe los volumenes de cada mes dependiendo del volumen deseado

for j in range(columnas):
  optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)) <= vecVolDeseado[j]
  #optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)) >= vecVolDeseado[j]-50000

# Restringe que el area maxima cosechable por mes

for j in range(columnas):
  if j<7:
    optCosecha += sum(area_a_cosechar[i][j] for i in range(filas)) >= 0
    optCosecha += sum(area_a_cosechar[i][j] for i in range(filas)) <= 0


#Restringe area de agosto para rendimiento de
#optCosecha += sum(area_a_cosechar[i][7] for i in range(filas)) >= 151
#optCosecha += sum(area_a_cosechar[i][7] for i in range(filas)) <= 225

#Restringe area de sep para rendimiento de 11700
#optCosecha += sum(area_a_cosechar[i][8] for i in range(filas)) >= 151
#optCosecha += sum(area_a_cosechar[i][8] for i in range(filas)) <= 240

#Restringe area de OCT para rendimiento de 11700
#optCosecha += sum(area_a_cosechar[i][9] for i in range(filas)) >= 151
#optCosecha += sum(area_a_cosechar[i][9] for i in range(filas)) <= 240

#Restringe area de nov para rendimiento de 11700
#optCosecha += sum(area_a_cosechar[i][10] for i in range(filas)) >= 151
#optCosecha += sum(area_a_cosechar[i][10] for i in range(filas)) <= 190

#Restringe area de dic para rendimiento de 11700
#optCosecha += sum(area_a_cosechar[i][11] for i in range(filas)) >= 151
#optCosecha += sum(area_a_cosechar[i][11] for i in range(filas)) <= 180



# -------------------------------------------------------------------------
# RESTRICCION MUISNE

#AGO
optCosecha += area_a_cosechar[8][7]+area_a_cosechar[9][7]+area_a_cosechar[10][7] >=0   #OCT PB56 0 [Ha]
optCosecha += area_a_cosechar[8][7]+area_a_cosechar[9][7]+area_a_cosechar[10][7] <=0   #OCT PB56 0 [Ha]
optCosecha += area_a_cosechar[6][7]+area_a_cosechar[7][7] >=0   #OCT PB55 0 [Ha]
optCosecha += area_a_cosechar[6][7]+area_a_cosechar[7][7] <=0   #OCT PB55 0 [Ha]

#SEP
optCosecha += area_a_cosechar[10][8]+area_a_cosechar[11][8]+area_a_cosechar[12][8] >=80.28   #SEP PB56 80.29 [Ha]
optCosecha += area_a_cosechar[10][8]+area_a_cosechar[11][8]+area_a_cosechar[12][8] <=80.30   #SEP PB56 80.29 [Ha]
optCosecha += area_a_cosechar[8][8]+area_a_cosechar[9][8] >=0   #OCT PB55 0 [Ha]
optCosecha += area_a_cosechar[8][8]+area_a_cosechar[9][8] <=0   #OCT PB55 0 [Ha]

#OCT
optCosecha += area_a_cosechar[10][9]+area_a_cosechar[11][9]+area_a_cosechar[12][9] >=65.74   #OCT PB56 65.75 [Ha]
optCosecha += area_a_cosechar[10][9]+area_a_cosechar[11][9]+area_a_cosechar[12][9] <=65.76   #OCT PB56 65.75 [Ha]
optCosecha += area_a_cosechar[8][9]+area_a_cosechar[9][9] >=48.55   #OCT PB55 48.56 [Ha]
optCosecha += area_a_cosechar[8][9]+area_a_cosechar[9][9] <=48.57   #OCT PB55 48.56 [Ha]

#NOV
optCosecha += area_a_cosechar[8][10]+area_a_cosechar[9][10] >=114.38   #OCT PB55 114.39 [Ha]
optCosecha += area_a_cosechar[8][10]+area_a_cosechar[9][10] <=114.40   #OCT PB55 114.39 [Ha]
optCosecha += area_a_cosechar[10][10]+area_a_cosechar[10][10]+area_a_cosechar[11][10] >=0   #OCT PB56 0 [Ha]
optCosecha += area_a_cosechar[10][10]+area_a_cosechar[10][10]+area_a_cosechar[11][10] <=0   #OCT PB56 0 [Ha]

#DIC
optCosecha += area_a_cosechar[10][11]+area_a_cosechar[11][11]+area_a_cosechar[12][11] >=79.34   #OCT PB56 79.35 [Ha]
optCosecha += area_a_cosechar[10][11]+area_a_cosechar[11][11]+area_a_cosechar[12][11] <=79.36   #OCT PB56 79.35 [Ha]
optCosecha += area_a_cosechar[8][11]+area_a_cosechar[9][11] >=0   #OCT PB55 0 [Ha]
optCosecha += area_a_cosechar[8][11]+area_a_cosechar[9][11] <=0   #OCT PB55 0 [Ha]


# RESTRICCION REMANENTE PB55 2020
optCosecha += area_a_cosechar[8][7]+area_a_cosechar[8][8]+area_a_cosechar[8][9]+area_a_cosechar[8][10]+area_a_cosechar[8][11] >=213.37-72.47
optCosecha += area_a_cosechar[8][7]+area_a_cosechar[8][8]+area_a_cosechar[8][9]+area_a_cosechar[8][10]+area_a_cosechar[8][11] <=213.37-72.45
#RESTRICCION NO REMANENTE  PB55 2021
optCosecha += area_a_cosechar[9][7]+area_a_cosechar[9][8]+area_a_cosechar[9][9]+area_a_cosechar[9][10]+area_a_cosechar[9][11] >= 44   #COSECHA TODO PB56
# RESTRICCION PB56 NO REMANENTE
optCosecha += area_a_cosechar[10][7]+area_a_cosechar[10][8]+area_a_cosechar[10][9]+area_a_cosechar[10][10]+area_a_cosechar[10][11] >=53   #COSECHA TODO PB56
optCosecha += area_a_cosechar[11][7]+area_a_cosechar[11][8]+area_a_cosechar[11][9]+area_a_cosechar[11][10]+area_a_cosechar[11][11] >=78   #COSECHA TODO PB56
optCosecha += area_a_cosechar[12][7]+area_a_cosechar[12][8]+area_a_cosechar[12][9]+area_a_cosechar[12][10]+area_a_cosechar[12][11] >=70   #COSECHA TODO PB56

#----------------------------------------------------------------------
#REMANENTE MUISNE 72.46

#optCosecha += area_a_cosechar[8][7]+area_a_cosechar[8][8]+area_a_cosechar[8][9]+area_a_cosechar[8][10]+area_a_cosechar[8][11]+area_a_cosechar[9][7]+area_a_cosechar[9][8]+area_a_cosechar[9][9]+area_a_cosechar[9][10]+area_a_cosechar[9][11]+ area_a_cosechar[10][7]+area_a_cosechar[10][8]+area_a_cosechar[10][9]+area_a_cosechar[10][10]+area_a_cosechar[10][11]+ area_a_cosechar[11][7]+area_a_cosechar[11][8]+area_a_cosechar[11][9]+area_a_cosechar[11][10]+area_a_cosechar[11][11]+area_a_cosechar[12][7]+area_a_cosechar[12][8]+area_a_cosechar[12][9]+area_a_cosechar[12][10]+area_a_cosechar[12][11] >= colAreaNeta[8]+colAreaNeta[9]+colAreaNeta[10]+colAreaNeta[11]+colAreaNeta[12]-72



#----------------------------------------------------------------------
# RESTRICCION MS09
optCosecha += area_a_cosechar[13][7]+area_a_cosechar[13][8]+area_a_cosechar[13][9]+area_a_cosechar[13][10]+area_a_cosechar[13][11] >=138   #COSECHA TODO MS09
# RESTRICCION PB63
optCosecha += area_a_cosechar[15][7]+area_a_cosechar[15][8]+area_a_cosechar[15][9]+area_a_cosechar[15][10]+area_a_cosechar[15][11] >=50.8   #COSECHA TODO PB63
# RESTRICCION PB64
optCosecha += area_a_cosechar[16][7]+area_a_cosechar[16][8]+area_a_cosechar[16][9]+area_a_cosechar[16][10]+area_a_cosechar[16][11] >=71.8   #COSECHA TODO PB64
# RESTRICCION PB51
optCosecha += area_a_cosechar[7][7]+area_a_cosechar[7][8]+area_a_cosechar[7][9]+area_a_cosechar[7][10]+area_a_cosechar[7][11] >= 174   #COSECHA TODO PB51
# RESTRICCION PB81
optCosecha += area_a_cosechar[17][7]+area_a_cosechar[17][8]+area_a_cosechar[17][9]+area_a_cosechar[17][10]+area_a_cosechar[17][11] >=117   #COSECHA TODO PB64
# RESTRICCION PB58
optCosecha += area_a_cosechar[14][7]+area_a_cosechar[14][8]+area_a_cosechar[14][9]+area_a_cosechar[14][10]+area_a_cosechar[14][11] >=38   #COSECHA TODO PB64


#-------------------------------------------------------------------------
'''
# FUERZA COSECHA FM16
optCosecha += area_a_cosechar[0][7]+area_a_cosechar[0][8]+area_a_cosechar[0][9]+area_a_cosechar[0][10]+area_a_cosechar[0][11] >=14
# FUERZA COSECHA FC06
optCosecha += area_a_cosechar[3][7]+area_a_cosechar[3][8]+area_a_cosechar[3][9]+area_a_cosechar[3][10]+area_a_cosechar[3][11] >=42
# FUERZA COSECHA MS11
optCosecha += area_a_cosechar[4][7]+area_a_cosechar[4][8]+area_a_cosechar[4][9]+area_a_cosechar[4][10]+area_a_cosechar[4][11] >=66
'''
#--------------------------------------------------------------------------


# 5) Restringe los rendimientos promedio por mes
#for j in range(columnas):
#  optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas))/sum(area_a_cosechar[i][j] for i in range(filas)) <= 10500
#  optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas))/sum(area_a_cosechar[i][j] for i in range(filas)) >= 10000

#----------------------------------------------------------

#Resolucion
optCosecha.solve()
#****************************** MODELO*********************************************************************************


#--------------Impresion de resultados preliminares--------------------------------------------------------------------
a=1
for j in range(filas): # recorre filas
  # Imprime cada sitio con el area asignada en el mes que se va a cosechar
  print(a, ") Areas ENE ", colSitioAnio[j], "-> " ,round(area_a_cosechar[j][0].value(),1), ", " ,round(area_a_cosechar[j][1].value(),1), ", " ,round(area_a_cosechar[j][2].value(),1),", " ,round(area_a_cosechar[j][3].value(),1),", " ,round(area_a_cosechar[j][4].value(),1),", " ,round(area_a_cosechar[j][5].value(),1),", " ,round(area_a_cosechar[j][6].value(),1),", " ,round(area_a_cosechar[j][7].value(),1), ", " ,round(area_a_cosechar[j][8].value(),1),", " ,round(area_a_cosechar[j][9].value(),1),", " ,round(area_a_cosechar[j][10].value(),1),", " ,round(area_a_cosechar[j][11].value(),1))
  a = a+1

for b in range(columnas):
  #Imprime el volumen a entregar cada mes
  print("Vol> ", b+1, "->", sum(matrizRend[i][b]*area_a_cosechar[i][b].value() for i in range(filas) ))

#Imprime el volumen total anual
print(sum(matrizRend[i][j]*area_a_cosechar[i][j].value() for j in range(columnas) for i in range(filas) ))
#-----------------------------------------------------------------------------------------------------------------------





1 ) Areas ENE  FM-162021 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  14.6
2 ) Areas ENE  MS-152020 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
3 ) Areas ENE  MS-162020 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  33.2
4 ) Areas ENE  PB-532021 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  31.9
5 ) Areas ENE  PR-052021 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
6 ) Areas ENE  FC-062021 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
7 ) Areas ENE  MS-112021 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
8 ) Areas ENE  PB-512021 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  24.0 ,  0.0 ,  150.0 ,  0.0
9 ) Areas ENE  PB-552020 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  6.3 ,  0.0 ,  4.6 ,  130.1 ,  0.0
10

Correccion de errores

In [332]:
# Matriz que obtiene los resultados de las areas optimizadas de cosecha
matrizAreaRes = [[0] * columnas for _ in range(filas)]
# Impresion de errores
countErrores = 0
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    matrizAreaRes [j][i] = area_a_cosechar[j][i].value()
    if area_a_cosechar[j][i].value() <0:
      countErrores = countErrores + 1
print("Errores-> ", countErrores)

volTotalRes = []
areaTotalRes = []
rendTotalRes = []
for j in range(columnas):

  volTotal = sum(matrizRendAux[i][j]*matrizAreaRes[i][j] for i in range(filas))
  volTotalRes.append(round(volTotal,1))
  areaTotal = sum(area_a_cosechar[i][j].value() for i in range(filas))
  areaTotalRes.append(round(areaTotal,1))
print("----------------------------------")
print("Volumen por mes")
print(volTotalRes)
print("Area a cosechar por mes")
print(areaTotalRes)

for k in range(len(volTotalRes)):
  if volTotalRes[k]>0:
    rendTotalRes.append(round(volTotalRes[k]/areaTotalRes[k],1))
  else:
    rendTotalRes.append(0)
print("Area a cosechar por mes")
print(rendTotalRes)
print("----------------------------------")

# Remanentes
for j in range(filas): # recorre filas
  if round(sum(matrizAreaRes[j]),2)<= round(colAreaNeta[j],2)-1:
    print(colSitioAnio[j], "-- ", round(sum(matrizAreaRes[j]),2),"--", round(colAreaNeta[j],2), "-- Remanente 2025 -- ",round(colAreaNeta[j],2)-round(sum(matrizAreaRes[j]),2))
  else:
    print(colSitioAnio[j], "-- ", round(sum(matrizAreaRes[j]),2),"--", round(colAreaNeta[j],2))

Errores->  0
----------------------------------
Volumen por mes
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2250000.0, 2500000.0, 2500000.0, 2000000.0, 1800000.0]
Area a cosechar por mes
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 165.4, 236.3, 209.0, 280.1, 167.7]
Area a cosechar por mes
[0, 0, 0, 0, 0, 0, 0, 13603.4, 10579.8, 11961.7, 7140.3, 10733.5]
----------------------------------
FM-162021 --  14.56 -- 14.56
MS-152020 --  0.0 -- 35.54 -- Remanente 2025 --  35.54
MS-162020 --  33.22 -- 33.22
PB-532021 --  31.9 -- 81.46 -- Remanente 2025 --  49.559999999999995
PR-052021 --  0.0 -- 4.02 -- Remanente 2025 --  4.02
FC-062021 --  0.0 -- 42.19 -- Remanente 2025 --  42.19
MS-112021 --  0.0 -- 66.55 -- Remanente 2025 --  66.55
PB-512021 --  174.0 -- 174.08
PB-552020 --  140.9 -- 213.37 -- Remanente 2025 --  72.47
PB-552021 --  44.0 -- 44.56
PB-562020 --  53.89 -- 53.89
PB-562020 --  78.93 -- 78.93
PB-562021 --  70.05 -- 70.05
MS-092020 --  138.6 -- 138.6
PB-582021 --  38.6 -- 38.6
PB-632021 --  50.81 -

Creacion del Excel con resultados


In [333]:
columnasRes = 13+12+12
matrizResultados = [[0] * columnasRes for _ in range(filas)]

#Asignación de nombres de sitio a la matriz de resultados
for h in range(filas):
  matrizResultados[h][0] = colSitioAnio[h]
k=1

# Asignación de resultados a la matriz
for k in range(columnasRes):
  for h in range(filas):
    if k>0 and k<=12:
      matrizResultados[h][k] = area_a_cosechar[h][k-1].value()
    if k>12 and k<=24:
      matrizResultados[h][k] = matrizRendAux[h][k-1-12]
    if k>24:
      matrizResultados[h][k] = matrizRendAux[h][k-1-24]*area_a_cosechar[h][k-1-24].value()

for f in range(filas):
  print(matrizResultados[f])

# Crea el Excel con los resultados,  areas por mes
#Nombres de las columnas del excel
nomColRes = ['Sitio-Año de plantación', 'Área ENE 2024', 'Área FEB 2024','Área MAR 2024', 'Área ABR 2024','Área MAY 2024','Área JUN 2024','Área JUL 2024','Área AGO 2024', 'Área SEP 2024','Área OCT 2024','Área NOV 2024','Área DIC 2024','Rendimiento ENE 2024','Rendimiento FEB 2024','Rendimiento MAR 2024','Rendimiento ABR 2024','Rendimiento MAY 2024','Rendimiento JUN 2024','Rendimiento JUL 2024','Rendimiento AGO 2024','Rendimiento SEP 2024','Rendimiento OCT 2024','Rendimiento NOV 2024','Rendimiento DIC 2024','VOL ENE 2024','VOL FEB 2024','VOL MAR 2024','VOL ABR 2024','VOL MAY 2024','VOL JUN 2024','VOL JUL 2024','VOL AGO 2024','VOL SEP 2024','VOL OCT 2024','VOL NOV 2024','VOL DIC 2024']
#Crea el data frame con los nombres de columnas y la matriz de resultados
df1 = pd.DataFrame(matrizResultados, columns=nomColRes)
# Path donde se crea el archivo resOptimizacion
pathExcel = '/content/drive/MyDrive/PB/resOptimizacion.xlsx'
#Guarda el data frame en el excel
df1.to_excel(pathExcel, index=False)


#Creacion de matriz de remanentes.
colRem = 4
matrizRemanentes = [[0] * colRem for _ in range(filas)]
for j in range(filas):
  matrizRemanentes[j][0] = colSitioAnio[j]
  matrizRemanentes[j][1] = sum(matrizAreaRes[j])
  matrizRemanentes[j][2] = colAreaNeta[j]
  matrizRemanentes[j][3] = colAreaNeta[j] - sum(matrizAreaRes[j])

print(matrizRemanentes)
#Titulos de remanentes
nomMatRem = ['Sitio-Año de plantación','Area a cosechar', 'Area neta Sitio', 'Remanente']
#Crea el data frame de remanentes
df2 =pd.DataFrame(matrizRemanentes, columns=nomMatRem)
pathExcel2 = '/content/drive/MyDrive/PB/remanenteOptimizacion.xlsx'
#Guarda el data frame en el excel
df2.to_excel(pathExcel2, index=False)


['FM-162021', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.56, 0, 0, 0, 0, 0, 0, 0, 0, 14628.821185829973, 15321.902640006409, 15968.801045790824, 16572.344878695254, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 241293.3414338029]
['MS-152020', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 9107.620802390013, 9658.655002210391, 10175.964072922125, 10661.032982425671, 11115.528792558605, 11541.19966910297, 11939.804673014993, 12313.066762794077, 12662.642889701048, 12990.106392500407, 13296.938030791607, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['MS-162020', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 33.22, 10416.298470703594, 10988.914527010053, 11525.309516822901, 12027.282738129385, 12496.77920843128, 12935.790847809843, 13346.289163491992, 13730.181451847338, 14089.284126270919, 14425.308207759705, 14739.853220700837, 15034.406712205928, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 499442.9909794809]
['P